In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_drug2_FL_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
data = pd.read_csv("./broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['drug_two_year'].values

In [4]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003,0.004,0.005]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

#### GAM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate = learning_rate,
                       depth = depth,
                       estimators=estimators,
                       holdout_split=holdout_split,
                       seed=816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia 

In [5]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.6538781478216074, 0.08366322579357326)

In [6]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.6808643542744772, 0.10732290145669496)

### Lasso Stumps

In [7]:
## load stumps & original data
data = pd.read_csv("./broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("./broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])

## split X & Y
X_stumps = data.loc[:,:'five_year1'].copy()
Y_stumps = data['drug_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("./broward/data/broward_train_stumps.csv").sort_values('person_id')
test_stumps = pd.read_csv("./broward/data/broward_test_stumps.csv").sort_values('person_id')
X_train_stumps = train_stumps.loc[:,:'five_year1'].copy()
X_test_stumps = test_stumps.loc[:,:'five_year1'].copy()
Y_train_stumps = train_stumps['drug_two_year'].values.copy()
Y_test_stumps = test_stumps['drug_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

#### Single Stump Model

In [8]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.08, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

29

In [9]:
single_stump_model['features']

['age_at_current_charge23',
 'age_at_current_charge27',
 'age_at_current_charge29',
 'age_at_current_charge31',
 'age_at_current_charge33',
 'age_at_current_charge57',
 'age_at_first_charge19',
 'age_at_first_charge21',
 'p_charges8',
 'p_probation2',
 'p_probation3',
 'p_felassault1',
 'p_weapon1',
 'p_fta_two_year1',
 'p_pending_charge2',
 'p_pending_charge6',
 'p_misdemeanor1',
 'p_misdemeanor4',
 'p_violence1',
 'p_violence5',
 'total_convictions2',
 'p_arrest3',
 'p_arrest4',
 'p_arrest6',
 'p_drug1',
 'p_drug2',
 'p_drug4',
 'p_dui1',
 'one_year1']

#### Nested Cross Validation

In [10]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05,
                                              0.06, 0.07, 0.08]},
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [11]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.08}, {'C': 0.07}, {'C': 0.08}, {'C': 0.08}, {'C': 0.08}],
 0.7176427384116442,
 0.07189410899365092)

#### Best Stump Model

In [12]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.08, 
                                      columns=cols, 
                                      seed=816)
best_stump_model['test_auc']

0.7099419334588827

### RiskSLIM

In [58]:
## load stumps data
data = pd.read_csv("./broward/data/broward_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.07,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

22

In [59]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['drug_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [60]:
riskslim_cs_summary = slim.risk_nested_cv_constrain(X=sub_X, 
                                                    Y=sub_Y, 
                                                    indicator = indicator,
                                                    y_label='drug_two_year', 
                                                    max_coef=5,
                                                    max_coef_number=5, 
                                                    max_runtime=1000, 
                                                    c=1e-6, 
                                                    max_offset=100,
                                                    seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
12/14/19 @ 01:47 PM | 273 rows in lookup table
12/14/19 @ 01:47 PM | ------------------------------------------------------------
12/14/19 @ 01:47 PM | runnning initialization procedure
12/14/19 @ 01:47 PM | ------------------------------------------------------------
12/14/19 @ 01:47 PM | CPA produced 2 cuts
12/14/19 @ 01:47 PM | running naive rounding on 21 solutions
12/14/19 @ 01:47 PM | best objective value: 0.2745
12/14/19 @ 01:47 PM | rounding produced 5 integer solutions
12/14/19 @ 01:47 PM | best objective value is 0.2852
12/14/19 @ 01:47 PM | running sequential rounding on 21 solutions
12/14/19 @ 01:47 PM | best objective value: 0.2745
12/14/19 @ 01:47 PM | sequential rounding produced 6 integer solutions
12/14/19 @ 01:47 PM | best objective value: 0.2715
12/14/19 @ 01:47 PM | polishing 11 solutions
12/14/19 @ 01:47 PM | best objective value: 0.2715
12/14/19 @ 01:47 PM | polishing produced 5 integer solutions
12/14/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 129 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2715.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 130 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2724.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 128 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2619.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 134 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2617.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 176 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2905.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 124 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2845.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:47 PM | adding 145 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2999.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:48 PM | adding 144 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2920.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable_ml\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [62]:
np.mean(riskslim_cs_summary['validation_auc']), np.mean(riskslim_cs_summary['test_auc'])

(0.6847243479649424, 0.6971674822777543)

#### Single RiskSLIM Model 

In [63]:
selected_features = ['drug_two_year'] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [64]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'drug_two_year',
    'sample_weights': sample_weights
}

In [65]:
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(new_train_data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
12/14/19 @ 01:48 PM | 273 rows in lookup table
12/14/19 @ 01:48 PM | ------------------------------------------------------------
12/14/19 @ 01:48 PM | runnning initialization procedure
12/14/19 @ 01:48 PM | ------------------------------------------------------------
12/14/19 @ 01:48 PM | CPA produced 2 cuts
12/14/19 @ 01:48 PM | running naive rounding on 26 solutions
12/14/19 @ 01:48 PM | best objective value: 0.2880
12/14/19 @ 01:48 PM | rounding produced 5 integer solutions
12/14/19 @ 01:48 PM | best objective value is 0.2914
12/14/19 @ 01:48 PM | running sequential rounding on 26 solutions
12/14/19 @ 01:48 PM | best objective value: 0.2880
12/14/19 @ 01:48 PM | sequential rounding produced 6 integer solutions
12/14/19 @ 01:48 PM | best objective value: 0.2914
12/14/19 @ 01:48 PM | polishing 11 solutions
12/14/19 @ 01:48 PM | best objective value: 0.2914
12/14/19 @ 01:48 PM | polishing produced 5 integer solutions
12/14/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
12/14/19 @ 01:48 PM | adding 159 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2914.
Tried aggregator 1 time.
Reduced MIP has 24 rows, 48 columns, and 91 nonzeros.
Reduced MIP has 22 binaries, 24 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [66]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [67]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7047066097339489

In [68]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7146892655367232

### Arnold PSA

In [49]:
### load data
data = pd.read_csv("./broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 
                       'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_two_year'].values

In [50]:
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')

In [51]:
np.mean(arnold['auc'])

0.552146277370336

#### Single Arnold PSA

In [52]:
test_data = pd.read_csv("./broward/data/broward_arnold_test.csv")
X = test_data['arnold_nca'].values
Y = test_data['drug_two_year'].values
roc_auc_score(Y, X)

0.6061676082862524

### Compass

In [53]:
### load data
data = pd.read_csv("./broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Recidivism_decile_score', 'sex', 'race', 'person_id', 
                       'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['drug_two_year'].values

In [55]:
compas = risktool.risktool(X_arnold, Y_arnold, label='Risk of Recidivism_decile_score')

In [57]:
np.mean(compas['auc'])

0.6326920829593098

### Results

In [69]:
#### save results
summary_drug2_FL_interpret = {"cart": cart_summary,
                              "ebm": ebm_summary, 
                              'stumps': stump_summary, 
                              'riskslim': riskslim_cs_summary, 
                              'arnold': arnold, 
                              'compas': compas}
%store summary_drug2_FL_interpret

Stored 'summary_drug2_FL_interpret' (dict)


In [70]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_cs_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])], 
           ['compas', np.mean(compas['auc'])]]
results

[['CART', 0.6538781478216074, 0.08366322579357326],
 ['EBM', 0.6808643542744772, 0.10732290145669496],
 ['Lasso Stumps', 0.7176427384116442, 0.07189410899365092],
 ['RiskSLIM-Constrained', 0.6971674822777543],
 ['Arnold PSA', 0.552146277370336],
 ['Compas', 0.6326920829593098]]

In [71]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_cs_summary['test_auc']),]

In [72]:
path = "./broward/logs/interpretable/"
results = [["Drug", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_cs_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_cs_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")", 
            np.str(round(np.mean(compas['auc']), 3)) + " (" + np.str(round(np.std(compas['auc']),3)) + ")"]]
with open(path + 'FL-two-year-interpretable-summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Save Fairness Matrix

In [9]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [13]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './broward/fairness/' + name[i] + '/two-year/drug/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_cs_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './broward/fairness/' + name[i] + '/two-year/drug/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_cs_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        compas_matrix = compas[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)
        compas_matrix.to_csv(path+'compas-'+name[i]+'.csv', index=None,header=True)